## Limpieza de archivos BibTeX

In [1]:
import os
import hashlib
import tempfile
import shutil
import re
from pybtex.database.input import bibtex as bibtex_input
from pybtex.database.output import bibtex as bibtex_output
from pybtex.database import BibliographyData
from natsort import natsorted

def process_bibtex_file_with_clean_environment(file_path):
    """Procesa un archivo BibTeX con un entorno limpio para evitar problemas de caché."""
    # Crear directorio temporal
    temp_dir = tempfile.mkdtemp()
    try:
        # Copiar el archivo a un directorio temporal con un nombre único
        temp_file = os.path.join(temp_dir, f"temp_{os.path.basename(file_path)}")
        shutil.copy2(file_path, temp_file)
        
        # Usar un nuevo parser para cada archivo
        parser = bibtex_input.Parser()
        bib_data = parser.parse_file(temp_file)
        
        return bib_data
    finally:
        # Limpiar el directorio temporal
        shutil.rmtree(temp_dir)

def normalize_title(title):
    """Normaliza un título para facilitar la comparación.
    Elimina espacios extra, signos de puntuación y convierte a minúsculas."""
    if not title:
        return ""
    # Eliminar caracteres especiales y convertir a minúsculas
    normalized = re.sub(r'[^\w\s]', '', title.lower())
    # Eliminar espacios múltiples y convertir a minúsculas
    normalized = re.sub(r'\s+', ' ', normalized).strip().lower()
    return normalized

def merge_bibtex_files(file_paths, output_path, duplicates_path):
    merged_db = BibliographyData()
    duplicates_list = []
    processed_titles = {}  # Cambiado de processed_ids a processed_titles
    duplicate_entries = set()  # Para contar entradas duplicadas únicas

    for file_path in file_paths:
        try:
            print(f"\nProcesando: {file_path}")
            bib_data = process_bibtex_file_with_clean_environment(file_path)

            for entry_id, entry in bib_data.entries.items():
                # Verificar si el entry tiene un campo de título
                if 'title' not in entry.fields:
                    print(f"  Advertencia: Entrada {entry_id} sin título en {file_path}, se agregará como única")
                    merged_db.add_entry(entry_id, entry)
                    continue
                
                # Normalizar el título para comparación
                title = entry.fields['title']
                normalized_title = normalize_title(title)
                
                if normalized_title in processed_titles:
                    # Encontramos un título duplicado
                    original_entry_id, original_file = processed_titles[normalized_title]
                    print(f"  Duplicado encontrado por título: {title}")
                    print(f"  Original ID: {original_entry_id} en: {original_file}")
                    print(f"  Duplicado ID: {entry_id} en: {file_path}")
                    
                    # Agregar el ID a la lista de duplicados únicos
                    duplicate_entries.add(entry_id)

                    # Guardar duplicado como texto en la lista
                    duplicates_list.append(f"@{entry.type}{{{entry_id},\n")
                    duplicates_list.append(f"  title = {{{title}}},\n")
                    for field, value in entry.fields.items():
                        if field != 'title':  # Ya agregamos el título
                            duplicates_list.append(f"  {field} = {{{value}}},\n")
                    duplicates_list.append("}\n\n")
                    
                else:
                    # Es un título nuevo, lo agregamos
                    merged_db.add_entry(entry_id, entry)
                    processed_titles[normalized_title] = (entry_id, file_path)

        except Exception as e:
            print(f"Error al procesar {file_path}: {e}")

    # Guardar el archivo consolidado
    writer = bibtex_output.Writer()
    writer.write_file(merged_db, output_path)
    print(f"Archivo consolidado guardado en: {output_path}")

    # Guardar el archivo de duplicados manualmente
    if duplicates_list:
        with open(duplicates_path, "w", encoding="utf-8") as f:
            f.writelines(duplicates_list)
        print(f"Duplicados guardados en: {duplicates_path}")
    else:
        print("No se encontraron duplicados.")
    
    return len(merged_db.entries), len(duplicate_entries)

def main():
    folder_path = os.getenv("DOWNLOAD_PATH") 

    # Obtenemos los archivos .bib sin la ruta completa
    files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.bib')]
    
    # Ordenamos usando natsorted para lograr un "orden natural"
    files_in_folder = natsorted(files_in_folder)

    # Verificar en que orden se procesaran los archivos
    print("Orden de procesamiento de archivos:")
    for i, f in enumerate(files_in_folder):
        print(f"{i+1}. {f}")

    # Construimos las rutas completas
    bibtex_files = [os.path.join(folder_path, f) for f in files_in_folder]

    # Verificar que los archivos sean únicos
    print("\nVerificando unicidad de archivos...")
    file_hashes = {}
    unique_files = []
    
    for file_path in bibtex_files:
        # Calcular hash del archivo
        file_hash = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b''):
                file_hash.update(chunk)
        
        # Verificar si ya hemos visto este hash
        digest = file_hash.hexdigest()
        if digest in file_hashes:
            print(f"¡ADVERTENCIA! Archivo duplicado detectado:")
            print(f"  - {file_path}")
            print(f"  - {file_hashes[digest]}")
            print(f"  Ambos tienen el mismo hash: {digest}")
        else:
            file_hashes[digest] = file_path
            unique_files.append(file_path)
    
    print(f"Total de archivos encontrados: {len(bibtex_files)}")
    print(f"Archivos únicos por contenido: {len(unique_files)}")
    
    # Proceder solo con archivos únicos
    bibtex_files = unique_files

    # Rutas para archivos de salida
    output_path = os.path.join( os.getenv("SALIDA_PATH") , "consolidado.bib")
    duplicates_path = os.path.join( os.getenv("DUPLICATE_PATH"), "duplicados.bib")
    
    unique_count, duplicate_count = merge_bibtex_files(bibtex_files, output_path, duplicates_path)
    
    print(f"\nResumen:")
    print(f"  Archivos procesados: {len(bibtex_files)}")
    print(f"  Entradas únicas: {unique_count}")
    print(f"  Entradas duplicadas: {duplicate_count}")

if __name__ == "__main__":
    main()

Orden de procesamiento de archivos:
1. IEEE Xplore Citation BibTeX Download 2025.9.24.22.54.21.bib
2. IEEE Xplore Citation BibTeX Download 2025.9.24.22.58.47.bib
3. IEEE Xplore Citation BibTeX Download 2025.9.24.23.0.57.bib
4. IEEE Xplore Citation BibTeX Download 2025.9.24.23.2.58.bib
5. IEEE Xplore Citation BibTeX Download 2025.9.24.23.17.14.bib
6. IEEE Xplore Citation BibTeX Download 2025.9.24.23.19.32.bib
7. IEEE Xplore Citation BibTeX Download 2025.9.24.23.32.17.bib
8. ScienceDirect_citations_1758818152466.bib
9. ScienceDirect_citations_1758818160849.bib
10. ScienceDirect_citations_1758818169328.bib
11. ScienceDirect_citations_1758818178033.bib
12. ScienceDirect_citations_1758818186439.bib
13. ScienceDirect_citations_1758818195117.bib
14. ScienceDirect_citations_1758818203568.bib
15. ScienceDirect_citations_1758818212043.bib
16. ScienceDirect_citations_1758818220494.bib
17. ScienceDirect_citations_1758818229059.bib
18. ScienceDirect_citations_1758818237681.bib
19. ScienceDirect_cit